In [1]:
""" Trainer for pretrain phase. """
import os.path as osp
import os
import numpy as np
import h5py
import tqdm
import datetime
import psutil
import torch
from torch.utils.data import DataLoader 
import torch.nn as nn
from dataset_loader import *
from utils.misc import Averager, Resize_Feature,Timer
#from utils.Tool import *
from Network_torch import *
from scipy import io
from testing import *

In [2]:
import argparse


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Basic parameters
    #####CHECKPOINT############################################################################################################ 
    parser.add_argument('--model_type', type=str, default='VGGDepthSingle256', choices=['UNet','VGGDepthSingle256','VGGDepth','VGGDepthSingle','VGG19Depth','VGGDepth1','VGGDepth1024', 'VGGDepth256','VGG16Attention','UNetDepth']) # The network architecture
    parser.add_argument('--loss_type', type=str, default='BCE', choices=['BCE', 'MSE']) # The loss function    

    # parser.add_argument('--phase', type=str, default='pre_train', choices=['pre_train', 'fine_tune', 'testing', 'testing_old']) # Phase    
    parser.add_argument('--ref_type', type=str, default='Constant', choices=['Constant','Reference'])# The Reference Velocity Type
    # parser.add_argument('--ref_num', type=int, default=6)# The Reference Velocity Number
    # parser.add_argument('--seed', type=int, default=0) # Manual seed for PyTorch, "0" means using random seed
    # parser.add_argument('--gpu', default='0') # GPU id
    parser.add_argument('--num_workers', type=int, default=1) # Number of CPU cores used for data loading
    
    # parser.add_argument('--Predthre', type=float, default=0.1)

    # parser.add_argument('--pre_dataset', type=str, default='COCO') # The pre-trained dataset
    # parser.add_argument('--pre_dataset_ver', type=str, default='COCO') # The pre-trained dataset version
    parser.add_argument('--pre_max_epoch', type=int, default=50) # Epoch number for pre-train phase
    parser.add_argument('--pre_batch_size', type=int, default=100) # Batch size for pre-train phase
    parser.add_argument('--pre_lr', type=float, default=0.0001) # Learning rate for pre-train phase
    parser.add_argument('--pre_gamma', type=float, default=0.1) # Gamma for the pre-train learning rate decay
    parser.add_argument('--pre_step_size', type=int, default=20) # The number of epochs to reduce the pre-train learning rate
    #####CHECKPOINT############################################################################################################
    parser.add_argument('--In_channels', type=int, default=1) # The number of epochs to reduce the pre-train learning rate
    parser.add_argument('--Out_channels', type=int, default=256) # The number of epochs to reduce the pre-train learning rate
    
    parser.add_argument('--pre_optimizer', type=str, default='Adam', choices=['Adam', 'sgd']) # optimizer for pre-train phase
    parser.add_argument('--pre_custom_momentum', type=float, default=0.9) # Momentum for the optimizer during pre-train    
    parser.add_argument('--pre_custom_weight_decay', type=float, default=0.0005) # Weight decay for the optimizer during pre-train
    parser.add_argument('--pre_init_weights', type=str, default=None)# The pre-trained weights for pre-train phase
    parser.add_argument('--pre_val_epoch', type=int, default=1) # The epoch interval for validation
    # parser.add_argument('--pre_early_stop', type=int, default=10) # The number of epochs to early stop during pre-train
    # parser.add_argument('--pre_early_stop_type', type=str, default='loss', choices=['loss','VMAE'])# The pre-trained weights for pre-train phase
    # parser.add_argument('--pre_dt', type=float, default=2, help='dt of the dataset, Unit: ms')
    # parser.add_argument('--pre_nt', type=int, default=1505, help='number of time sampling points')


    # Parameters for testing
    #####CHECKPOINT############################################################################################################
    parser.add_argument('--test_dataset_dir', type=str, default='./')# The pre-trained dataset folder
    parser.add_argument('--test_dataset_name', type=str, default='Depth_Single_station_CESO') # The pre-trained dataset
    parser.add_argument('--pre_input_data', type=str, default='testing_data/station3_input.hdf5')
    #parser.add_argument('--pre_label_data', type=str, default='Test_label_depth_17-05-1.3.hdf5')

    parser.add_argument('--test_batch_size', type=int, default=100) # Batch size for testing
    ## CHECKPOINT############################################################################################################
    parser.add_argument('--test_init_weights', type=str, default='./well_trained_model/CESI_epoch_300.pth')# The pre-trained weights for testing
    parser.add_argument('--test_height', type=int, default=96)
    parser.add_argument('--test_width', type=int, default=192)






    args = parser.parse_args([])

    # 在这里可以使用 args 来访问解析后的参数
    print(args)

Namespace(model_type='VGGDepthSingle256', loss_type='BCE', ref_type='Constant', num_workers=1, pre_max_epoch=50, pre_batch_size=100, pre_lr=0.0001, pre_gamma=0.1, pre_step_size=20, In_channels=1, Out_channels=256, pre_optimizer='Adam', pre_custom_momentum=0.9, pre_custom_weight_decay=0.0005, pre_init_weights=None, pre_val_epoch=1, test_dataset_dir='./', test_dataset_name='Depth_Single_station_CESO', pre_input_data='testing_data/station3_input.hdf5', test_batch_size=100, test_init_weights='./well_trained_model/CESI_epoch_300.pth', test_height=96, test_width=192)


### Depth Prediction

In [3]:
# instance = DepthTestingLearner(args)   # Create an instance of the class
# train = instance.test() # Start training

### Single Station Prediction

In [4]:
instance = SingleStationDepthTestingLearner(args)   # Create an instance of the class
train = instance.test()  # Start training

The number of testing samples is 6612
The size of testing samples is 96 x 192


100%|██████████| 67/67 [00:01<00:00, 49.30it/s]

Test is completed!
The result is saved in ./logs_fortest/test/Depth_Single_station_CESO_lr0.0001_batch100


In [5]:
# instance = TestingLearner(args)   # Create an instance of the class
# train = instance.test() # Start training